# Linear programming to solve an objective function

In [ ]:
import scipy.optimize as spopt

c = [10,30]
A = [[2,-1]]
b = [0]
r1_bound = (10,None)
r7_bound = (10,None)
res = spopt.linprog(c,A_ub=A,b_ub=b, 
                    bounds=(r1_bound,r7_bound), method='simplex')
print(res)

In [ ]:
import scipy.optimize as spopt

c = [-0.03,-0.02,-0.001,-0.001]
A = [[1,1,0,0],[0,0,2,1]]
b = [20000, 30000]
x1_bound = (0,2*10**6)
x2_bound = (0,4*10**6)
x3_bound = (0,2*10**6)
x4_bound = (0,4*10**6)
res = spopt.linprog(c,A_ub=A,b_ub=b, 
                          bounds=(x1_bound,x2_bound,x3_bound,x4_bound), 
                    method='simplex')
print(res)

# Finite difference examples

In [ ]:
def finite_diff(f,x):
    dx = 1e-10
    return((f(x+dx)-f(x-dx))/(2*dx))

f = lambda x: x**2

finite_diff(f,4.0)

In [ ]:
def finite_diff(f,x,y):
    dx = dy = 1e-10
    return( (f(x+dx,y+dy)-f(x,y))/((dx+dy)/2) )

f = lambda x, y: y**2 + 10*(x/y)

finite_diff(f,20,10)

In [ ]:
def finite_diff_xy(f, x, y, dx, dy):
    
    return( (f(x+dx,y+dy)-f(x-dx,y-dy))/(2*(dx+dy)) )

f = lambda x, y: y**2 + (10*(x/y))

finite_diff_xy(f,20,10,0,1e-10)

## compute jacobian

In [ ]:
import numpy as np

def finite_diff_xy(f, x, y, dx, dy):
    return( (f(x+dx,y+dy)-f(x-dx,y-dy))/(2*(dx+dy)) )

f1 = lambda x, y: y**2 + (10*(x/y))
f2 = lambda x, y: 10*x + 30*y

J = np.array([[finite_diff_xy(f1,20,10,1e-10,0),finite_diff_xy(f1,20,10,0,1e-10)],
              [finite_diff_xy(f2,20,10,1e-10,0),finite_diff_xy(f2,20,10,0,1e-10)]])

J

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.figure()
x = np.maximum(25,np.abs(np.arange(-100,100,0.1)))**2
ax = sns.lineplot(np.arange(-100,100,0.1),x)
ax.set(xlabel="x",ylabel="function")
plt.savefig('multiple_minima.png',dpi=300)

plt.figure()
x = np.abs(np.arange(-100,100,0.1))**2
ax = sns.lineplot(np.arange(-100,100,0.1),x)
ax.set(xlabel="x",ylabel="function")
plt.savefig('single_minima.png',dpi=300)

In [ ]:
x = np.arange(-100,100,0.5)
y = np.arange(-100,100,0.5)
xx,yy = np.meshgrid(x,y)
z = xx**2 - yy**2
fig, ax = plt.subplots()
cx = plt.contourf(xx,yy,z)
ax.set(xlabel="x",ylabel="y")
fig.colorbar(cx)
plt.savefig('saddle_points.png',dpi=300)

# Optimization of an objective

1. Grid Search
2. Gradient descent
3. Momentum based gradient descent
4. Newtons method

## Grid Search

In [ ]:
import numpy as np

x = np.arange(-50,50,10)
y = np.arange(-50,50,10)
xx,yy = np.meshgrid(x,y)
z = np.ones(xx.shape)
plt.scatter(xx,yy)
plt.xlim(-100,100)
plt.ylim(-100,100)
plt.savefig('gridsearch-2.png',dpi=300)

In [ ]:
def cost(R1,R7):
    if R1<10 or R7 <10 or (2*R1-R7)<=0:
        return(np.inf)
    else:
        return(10*R1+30*R7)

min_val = np.inf
# search the grid of values between -50 and 50
# with 100 points for each parameter
for R1 in range(-50,50):
    for R7 in range(-50,50):
        if min_val > cost(R1,R7):
            param = (R1,R7)
            min_val = cost(R1,R7)

print(param, min_val)

## Gradient Descent

In [ ]:
# utility functions for visualization
def plot_contour(support, f, levels):
    x = np.arange(-support,support,0.05)
    y = np.arange(-support,support,0.05)
    xx,yy = np.meshgrid(x,y)
    vecs = np.stack((xx,yy))
    z = f(vecs) 
    logz = np.log(z)
    fig, ax = plt.subplots(figsize=(12,10))
    cx = plt.contour(xx,yy,logz, levels=levels, cmap='tab10')
    ax.set_xlabel("x", fontsize=32)
    ax.set_ylabel("y", fontsize=32)
    ax.tick_params(axis='both', which='major', labelsize=16)
    plt.xlim(-support,support)
    plt.ylim(-support,support)
    cbar = fig.colorbar(cx)
    cbar.ax.tick_params(labelsize=16)
    return(fig)
    
def add_grads(fnum, x_updates, grads, scale, color):
#     fig = plt.figure(fnum)
    q = plt.quiver(x_updates[:-1,0],x_updates[:-1,1],-grads[1:,0],-grads[1:,1], scale=scale, width=0.005, 
               alpha=0.5, color=color)
    return(q)
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# cost function
def cost(R1,R7):
    return(R1**2+R7**2)

def plot_cost(x):
    return(x[0]**2+x[1]**2)


# finite difference approximation
def finite_diff_xy(f, x, y, dx, dy):
    return( (f(x+dx,y+dy)-f(x-dx,y-dy))/(2*(dx+dy)) )

# gradient calculation using finite difference
def gradient(f, x):
    dx = dy = 1e-10
    return(np.array([ finite_diff_xy(f,x[0],x[1],dx,0), 
                      finite_diff_xy(f,x[0],x[1],0,dy)]) )

# gradient descent to find minimum value
def gradient_descent(gradient, f, x, eta):
    x_prev = np.array(x)
    x_new = np.zeros(x_prev.shape)
    # learning rate/step size
    eta = eta
    # store the x values and gradients at each step in x_updates and grads
    x_updates = x_prev
    grads = np.zeros(x_prev.shape)
    
    # initalize tolerance and threshold for termination
    tolerance = 1e-10
    threshold = 1

    # loop until threshold is under tolerance
    while(threshold > tolerance):
        # calculate gradient
        grad = gradient(f, x_prev)
        # take a step in opposite direction of gradient
        x_new = x_prev - (eta*grad)
        # calcualte threshold and update the parameters
        threshold = np.linalg.norm(x_prev-x_new)
        x_prev = x_new
        # store the gradients and updated parameters
        grads = np.vstack((grads,grad))
        x_updates = np.vstack((x_updates,x_prev))
    return(x_new,x_updates, grads)

x_new, x_updates, grads = gradient_descent(gradient, cost, np.array([4,6]), 0.1)

# plot the function and gradients to reach minimum
support = 10
levels = 100
fig = plot_contour(support, plot_cost, levels)
grad = add_grads(fig.number, x_updates, grads, 200, 'red')

plt.savefig('gradient_descent.png',dpi=300)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def cost(R1,R7):
    return(R1**2+R7**2)

def plot_cost(x):
    return(x[0]**2+x[1]**2)


def finite_diff_xy(f, x, y, dx, dy):
    return( (f(x+dx,y+dy)-f(x-dx,y-dy))/(2*(dx+dy)) )

def gradient(f, x):
    dx = dy = 1e-10
    return(np.array([ finite_diff_xy(f,x[0],x[1],dx,0), 
                      finite_diff_xy(f,x[0],x[1],0,dy)]) )

# start at (4,6)
def gradient_descent(gradient, f, x, eta):
    x_prev = np.array(x)
    x_new = np.zeros(x_prev.shape)
    # learning rate/step size
    eta = eta
    x_updates = x_prev
    grads = np.zeros(x_prev.shape)
    tolerance = 1e-10
    threshold = 1

    while(threshold > tolerance):
        grad = gradient(f, x_prev)

        x_new = x_prev - (eta*grad)
        threshold = np.linalg.norm(x_prev-x_new)
        x_prev = x_new
        # store the gradients and updated parameters
        grads = np.vstack((grads,grad))
        x_updates = np.vstack((x_updates,x_prev))
    return(x_new,x_updates, grads)

x_new, x_updates, grads = gradient_descent(gradient, cost, np.array([4,6]), 0.1)

support = 10
levels = 100
fig = plot_contour(support, plot_cost, levels)
grad = add_grads(fig.number, x_updates[0:2,:], -grads[0:2,:], 200, 'red')
grad = add_grads(fig.number, x_updates[0:2,:], grads[0:2,:], 200, 'blue')

plt.savefig('gradient_descent_direction.png',dpi=300)
x_new

## momentum illustration using the booth function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# cost function - booth function
def cost(R1,R7):
    return(np.power(R1+2*R7-7,2)+np.power(2*R1+R7-5,2))

def plot_cost(x):
    return(np.power(x[0]+2*x[1]-7,2)+np.power(2*x[0]+x[1]-5,2))


# finite difference approximation
def finite_diff_xy(f, x, y, dx, dy):
    return( (f(x+dx,y+dy)-f(x-dx,y-dy))/(2*(dx+dy)) )

# gradient calculation using finite difference
def gradient(f, x):
    dx = dy = 1e-10
    return(np.array([ finite_diff_xy(f,x[0],x[1],dx,0), 
                      finite_diff_xy(f,x[0],x[1],0,dy)]) )

# gradient descent to find minimum value
def gradient_descent(gradient, f, x, eta):
    x_prev = np.array(x)
    x_new = np.zeros(x_prev.shape)
    # learning rate/step size
    eta = eta
    # store the x values and gradients at each step in x_updates and grads
    x_updates = x_prev
    grads = np.zeros(x_prev.shape)
    
    # initalize tolerance and threshold for termination
    tolerance = 1e-10
    threshold = 1

    # loop until threshold is under tolerance
    while(threshold > tolerance):
        # calculate gradient
        grad = gradient(f, x_prev)
        # take a step in opposite direction of gradient
        x_new = x_prev - (eta*grad)
        # calcualte threshold and update the parameters
        threshold = np.linalg.norm(x_prev-x_new)
        x_prev = x_new
        # store the gradients and updated parameters
        grads = np.vstack((grads,grad))
        x_updates = np.vstack((x_updates,x_prev))
    return(x_new,x_updates, grads)



def momentum_gradient(gradient, f, x, eta, alpha):
    x_prev = np.array(x)
    x_new = np.zeros(x_prev.shape)
    # learning rate/step size
    eta = eta
    # momentum
    alpha = alpha
    # store the x values and gradients at each step in x_updates and grads
    x_updates = x_prev
    grads = np.zeros(x_prev.shape)
    
    # initalize tolerance and threshold for termination
    tolerance = 1e-10
    threshold = 1

    # loop until threshold is under tolerance
    while(threshold > tolerance):
        # calculate gradient
        grad = gradient(f, x_prev)
        # take a step in opposite direction of gradient
        if len(grads.shape) > 1:
            grad_momentum = (eta*grad) + (alpha*grads[-1,:])
        else:
            grad_momentum = (eta*grad)
        x_new = x_prev - grad_momentum
        # calcualte threshold and update the parameters
        threshold = np.linalg.norm(x_prev-x_new)
        x_prev = x_new
        # store the gradients and updated parameters
        grads = np.vstack((grads,grad_momentum))
        x_updates = np.vstack((x_updates,x_prev))
    return(x_new,x_updates, grads)

x_new, x_updates, grads = gradient_descent(gradient, cost, np.array([10,0]), 0.1)
x_new, momentum_updates, momentum_grads = momentum_gradient(gradient, cost, np.array([10,0]), 0.1, 0.2)

print(grads.shape, momentum_grads.shape)

# plot the function and gradients to reach minimum
support = 10
levels = 100
fig = plot_contour(support, plot_cost, levels)
grad = add_grads(fig.number, x_updates, grads, 200, 'red')
momentum = add_grads(fig.number, momentum_updates, momentum_grads, 20, 'blue')

plt.legend((grad,momentum),('Gradient','Momentum'),prop={'size': 16})
plt.savefig('gradient_descent_momentum.png',dpi=300)

## Newton's Method

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

f = lambda x : np.power(x[0]**2+x[1]-11,2)+np.power(x[0]+x[1]**2-7,2)
support = 7
fig = plot_contour(support, f, 140)
plt.show(fig)
plt.savefig('himmelbaus.png',dpi=300)

In [ ]:
import autograd.numpy as np
from autograd import grad


# cost function - booth function
def cost(x):
    return(np.power(x[0]+2*x[1]-7,2)+np.power(2*x[0]+x[1]-5,2))

grad_f = grad(cost)


def gradient_descent(gradient, x, eta):
    x_prev = np.array(x)
    x_new = np.zeros(x_prev.shape)
    # learning rate/step size
    eta = eta
    x_updates = x_prev
    grads = np.zeros(x_prev.shape)
    tolerance = 1e-10
    threshold = 1
    iteration = 1
    
    while(threshold > tolerance):
        grad = gradient(x_prev)
        # decrease step size as iteration go up
        x_new = x_prev - (eta*grad)
        threshold = np.linalg.norm(x_prev-x_new)
        x_prev = x_new
        # store the gradients and updates
        grads = np.vstack((grads,grad))
        x_updates = np.vstack((x_updates,x_prev))
        iteration += 1
        if iteration%1000==0:
            print("gradient iteration:",iteration)
    return(x_new,x_updates, grads)

from autograd import hessian
hess_f = hessian(cost)

def newtons_method(gradient, hessian, x, eta):
    x_prev = np.array(x)
    x_new = np.zeros(x_prev.shape)
    # learning rate/step size
    eta = eta
    x_updates = x_prev
    steps = np.zeros(x_prev.shape)
    tolerance = 1e-10
    threshold = 1
    iteration = 1
    
    while(threshold > tolerance):
        grad = gradient(x_prev)
        hess = hessian(x_prev)
        # calculate step direction
        step = np.matmul(np.linalg.inv(hess), grad)
        # decrease step size
        x_new = x_prev - (eta*step)
        threshold = np.linalg.norm(x_prev-x_new)
        x_prev = x_new
        # store the gradients and updated parameters
        steps = np.vstack((steps,step))
        x_updates = np.vstack((x_updates,x_prev))
        iteration += 1
        if iteration%1000==0:
            print("newton iteration:",iteration)
    return(x_new,x_updates, steps)


support = 12
fig = plot_contour(support, cost, 50)

x_new_grad, x_updates, grads = gradient_descent(grad_f, np.array([10.0,-10.0]), 0.1)
x_new, newton_updates, steps = newtons_method(grad_f,hess_f,np.array([10.0,-10.0]), 0.1)
print("gradient_steps=", x_updates.shape, x_new_grad)
print("newton_steps=", newton_updates.shape, x_new)
gradient_2 = add_grads(fig.number, x_updates, grads, 200, 'green')
newton_2 = add_grads(fig.number, newton_updates, steps, 200, 'cyan')


x_new_grad, x_updates, grads = gradient_descent(grad_f, np.array([10.0,-10.0]), 0.2)
x_new, newton_updates, steps = newtons_method(grad_f,hess_f,np.array([10.0,-10.0]), 0.2)
print("gradient_steps=",x_updates.shape, x_new_grad)
print("newton_steps=",newton_updates.shape, x_new)
gradient = add_grads(fig.number, x_updates, grads, 200, 'red')
newton = add_grads(fig.number, newton_updates, 1*steps, 200, 'blue')


plt.legend((gradient,newton,gradient_2,newton_2),('Gradient(eta=0.2)','newton(eta=0.2)',
                                                  'gradient(eta=0.1)','newton(eta=0.1)'),prop={'size': 16})
plt.savefig('newtons_method.png',dpi=300)
plt.show(fig)

## automatic differentation

In [ ]:
from autograd import grad
from autograd import numpy as np
# cost function
def cost(x):
    return(x[1]**2+(10*x[0]/x[1]))

gradient = grad(cost)
gradient_val = gradient(np.array([20.0,10.0]))
print(gradient_val)

# dill function for activity

In [ ]:
import dill

def fun(x):
    import numpy as np
    np.random.seed(100)
    val = np.power( (np.power(x[0],2) + x[1]-11) ,2) + np.power( (x[0]+ np.power(x[1],2)-7) ,2) #+ np.random.normal(loc=-1.0,scale=5)
    return(val)

file = open('function_activity1','wb')
dill.dump(fun,file)

In [ ]:
# import dill
# from inspect import signature
# function = open('function_activity1','rb')

# # load function using dill package
# f = dill.load(function)
# sig = signature(f)
# str(sig)
# f([10,20])

In [ ]:
import autograd.numpy as np
import matplotlib.pyplot as plt
from autograd import grad
from autograd import hessian
import dill
%matplotlib inline

function = open('function_activity1','rb')

# load function using dill package
f = dill.load(function)

# calculate gradient using autograd
grad_f = grad(f)

# perform gradient descent 
def gradient_descent(gradient, x, eta):
    x_prev = np.array(x)
    x_new = np.zeros(x_prev.shape)
    # learning rate/step size
    eta = eta
    x_updates = x_prev
    grads = np.zeros(x_prev.shape)
    tolerance = 1e-10
    threshold = 1
    iteration = 1
    
    while(threshold > tolerance):
        grad = gradient(x_prev)
        # decrease step size as iteration go up
        x_new = x_prev - (eta*grad)
        threshold = np.linalg.norm(x_prev-x_new)
        x_prev = x_new
        # store the gradients and updates
        grads = np.vstack((grads,grad))
        x_updates = np.vstack((x_updates,x_prev))
        iteration += 1
    return(x_new,x_updates, grads)

x1_new, x_updates, grads = gradient_descent(grad_f, np.array([-10.0,10.0]),0.001)
print("Search terminated")
support = 10
print("Best Parameters from gradient descent:",x1_new)
print("steps for gradient descent",x_updates.shape[0])

fig = plot_contour(support, f, 100)
grad = add_grads(fig.number, x_updates, grads, 2*10**4, 'red')
plt.savefig('gradient_descent_auto.png',dpi=300)
plt.show(fig)
grads

In [ ]:
x = np.arange(-support,support,0.05)
y = np.arange(-support,support,0.05)
xx,yy = np.meshgrid(x,y)
vecs = np.stack((xx,yy))
z = f(vecs) 
logz = np.log(z)
np.min(logz)

In [ ]:
import autograd.numpy as np
import matplotlib.pyplot as plt
from autograd import grad
from autograd import hessian
import dill
%matplotlib inline

function = open('function_activity1','rb')
f = dill.load(function)

grad_f = grad(f)
hess_f = hessian(f)

def gradient_descent(gradient, x, eta):
    x_prev = np.array(x)
    x_new = np.zeros(x_prev.shape)
    # learning rate/step size
    eta = eta
    x_updates = x_prev
    grads = np.zeros(x_prev.shape)
    tolerance = 1e-10
    threshold = 1
    iteration = 1
    
    while(threshold > tolerance):
        grad = gradient(x_prev)
        # decrease step size as iteration go up
        x_new = x_prev - (eta*grad)
        threshold = np.linalg.norm(x_prev-x_new)
        x_prev = x_new
        # store the gradients and updates
        grads = np.vstack((grads,grad))
        x_updates = np.vstack((x_updates,x_prev))
        iteration += 1
        if iteration%1000==0:
            print("newton iteration:",iteration)
    return(x_new,x_updates, grads)

x1_new, x_updates, grads = gradient_descent(grad_f, np.array([-10.0,10.0]),0.001)
x2_new, newton_updates, steps = newtons_method(grad_f,hess_f,np.array([-10.0,10.0]), 0.8)
print("terminated")
support = 7
print("Best Parameters from gradient descent:",x1_new)
print("Best Parameters from newtons method:",x2_new)

print("steps for gradient descent",x_updates.shape[0])
print("steps for newtons method", newton_updates.shape[0])
fig = plot_contour(support, f, 140)
grad = add_grads(fig.number, x_updates, grads, 10000, 'red')
newton = add_grads(fig.number, newton_updates, steps, 50, 'blue')
plt.show(fig)
plt.savefig('newtown_auto.png',dpi=300)

## Maximum likelihood

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
data = np.random.normal(1000, 100,1000) + np.random.normal(100,20,1000)

sns.distplot(data, kde=False)

# Simplex

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fmin
%matplotlib inline

def cost(x):
    return(np.power(x[0]+2*x[1]-7,2)+np.power(2*x[0]+x[1]-5,2))

# fmin uses nelder-mead but has the option to return all vectors for analysis unlike minimize
"""
res_min = scipy.optimize.minimize(cost,np.array([-10,10]), method='Nelder-Mead')

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 49
         Function evaluations: 94
 final_simplex: (array([[0.99997756, 3.00002429],
       [1.00003059, 3.00000105],
       [1.00006341, 2.99995153]]), array([1.10730863e-09, 4.94047525e-09, 7.26423839e-09]))
           fun: 1.1073086283232063e-09
       message: 'Optimization terminated successfully.'
          nfev: 94
           nit: 49
        status: 0
       success: True
             x: array([0.99997756, 3.00002429])
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 49
         Function evaluations: 94

"""



result = fmin(cost, np.array([-10,10]), retall=True)
print(result[0])
res = np.array(result[1])
support = 15
fig = plot_contour(support, cost, 140)
grad = add_grads(fig.number, res, 3*np.sign(res), 100, 'red')
plt.savefig('Nelder-mead.png',dpi=300)


# Differential Evolution

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import differential_evolution
import dill
%matplotlib inline

function = open('function_activity1','rb')
f = dill.load(function)

def cost(x):
    return(np.power(x[0]+2*x[1]-7,2)+np.power(2*x[0]+x[1]-5,2))

bounds = [(-12,12),(-12,12)]
result = differential_evolution(f, bounds, popsize=5, mutation=(0.5,0.8))
result

# Cross Validation

In [ ]:
import seaborn as sns
import numpy as np
%matplotlib inline

# sns.scatterplot(data[:,0],data[:,1],sizes=[1000])

# utility functions for visualization
def plot_contourf(support, f, levels):
    x = np.arange(support[0],support[1],0.05)
    y = np.arange(support[0],support[1],0.05)
    xx,yy = np.meshgrid(x,y)
    vecs = np.stack((xx,yy))
    z = f(vecs) 
    #logz = np.abs(np.log(z))
    logz = np.log(z)
    fig, ax = plt.subplots(figsize=(12,10))
    cx = plt.contourf(xx,yy,logz, levels=levels, cmap='tab10')
    ax.set_xlabel("x", fontsize=32)
    ax.set_ylabel("y", fontsize=32)
    ax.tick_params(axis='both', which='major', labelsize=16)
    plt.xlim(support[0],support[1])
    plt.ylim(support[0],support[1])
    cbar = fig.colorbar(cx)
    cbar.ax.tick_params(labelsize=16)
    return(fig)

def f(x):
    result = np.power((x[0]-10),2)+np.power((x[1]-10),2)
    return(result)

support = [0,20]
f = plot_contourf(support,f,4)

np.random.seed(10)
cov1 = np.array(([0.1,0],[0,0.1]))
mean1 = np.array([13,13])
data = np.random.multivariate_normal(mean1, cov1,10)
np.min(data)
plt.scatter(data[:,0],data[:,1],color='b', alpha=0.5, linewidths=[5])
plt.savefig('high-low.png',dpi=300)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
iris = datasets.load_iris()
print("Dataset dimensions:",iris.data.shape)
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=100)
print("Training set dimensions:",X_train.shape)
print("Testing set dimensions:", X_test.shape)

Dataset dimensions: (150, 4)
Training set dimensions: (100, 4)
Testing set dimensions: (50, 4)


In [3]:
from sklearn.model_selection import LeaveOneOut
from sklearn import datasets
iris = datasets.load_iris()
print("Dataset dimensions:",iris.data.shape)
loo = LeaveOneOut()
count = 1
for train_index, test_index in loo.split(iris.data):
    print("{2}.\tTraining set:{0} \n\tTesting set:{1}".format(train_index.shape,test_index.shape,count))
    count += 1

Dataset dimensions: (150, 4)
1.	Training set:(149,) 
	Testing set:(1,)
2.	Training set:(149,) 
	Testing set:(1,)
3.	Training set:(149,) 
	Testing set:(1,)
4.	Training set:(149,) 
	Testing set:(1,)
5.	Training set:(149,) 
	Testing set:(1,)
6.	Training set:(149,) 
	Testing set:(1,)
7.	Training set:(149,) 
	Testing set:(1,)
8.	Training set:(149,) 
	Testing set:(1,)
9.	Training set:(149,) 
	Testing set:(1,)
10.	Training set:(149,) 
	Testing set:(1,)
11.	Training set:(149,) 
	Testing set:(1,)
12.	Training set:(149,) 
	Testing set:(1,)
13.	Training set:(149,) 
	Testing set:(1,)
14.	Training set:(149,) 
	Testing set:(1,)
15.	Training set:(149,) 
	Testing set:(1,)
16.	Training set:(149,) 
	Testing set:(1,)
17.	Training set:(149,) 
	Testing set:(1,)
18.	Training set:(149,) 
	Testing set:(1,)
19.	Training set:(149,) 
	Testing set:(1,)
20.	Training set:(149,) 
	Testing set:(1,)
21.	Training set:(149,) 
	Testing set:(1,)
22.	Training set:(149,) 
	Testing set:(1,)
23.	Training set:(149,) 
	Testing 

In [5]:
from sklearn.model_selection import KFold
from sklearn import datasets
iris = datasets.load_iris()
print("Dataset dimensions:",iris.data.shape)
kf = KFold(n_splits=3)
kf.get_n_splits(iris.data)
count = 1
print("\nK-Fold cross validation\n")
for train_index, test_index in kf.split(iris.data):
    print("{2}.\tTraining set:{0} \n\tTesting set:{1}".format(train_index.shape,test_index.shape,count))
    count += 1

Dataset dimensions: (150, 4)

K-Fold cross validation

1.	Training set:(100,) 
	Testing set:(50,)
2.	Training set:(100,) 
	Testing set:(50,)
3.	Training set:(100,) 
	Testing set:(50,)


In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn import datasets
from collections import Counter

iris = datasets.load_iris()
print("Dataset dimensions:",iris.data.shape)

print("\nK-Fold cross validation\n")
kf = KFold(n_splits=3)
kf.get_n_splits(iris.data)
count = 1
for train_index, test_index in kf.split(iris.data):
    train_count = Counter(iris.target[train_index])
    test_count = Counter(iris.target[test_index])
    print("{2}.\tTraining set:{0} \n\tTesting set:{1}".format(train_count,test_count,count))
    count += 1

print("\nStratified K-Fold cross validation\n")
skf = StratifiedKFold(n_splits=3)
skf.get_n_splits(iris.data,iris.target)
count = 1    
for train_index, test_index in skf.split(iris.data, iris.target):
    train_count = Counter(iris.target[train_index])
    test_count = Counter(iris.target[test_index])
    print("{2}.\tTraining set:{0} \n\tTesting set:{1}".format(train_count,test_count,count))
    count += 1

Dataset dimensions: (150, 4)

K-Fold cross validation

1.	Training set:Counter({1: 50, 2: 50}) 
	Testing set:Counter({0: 50})
2.	Training set:Counter({0: 50, 2: 50}) 
	Testing set:Counter({1: 50})
3.	Training set:Counter({0: 50, 1: 50}) 
	Testing set:Counter({2: 50})

Stratified K-Fold cross validation

1.	Training set:Counter({0: 33, 1: 33, 2: 33}) 
	Testing set:Counter({0: 17, 1: 17, 2: 17})
2.	Training set:Counter({0: 33, 1: 33, 2: 33}) 
	Testing set:Counter({0: 17, 1: 17, 2: 17})
3.	Training set:Counter({0: 34, 1: 34, 2: 34}) 
	Testing set:Counter({0: 16, 1: 16, 2: 16})


In [8]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()

X_train = iris.data
y_train = iris.target
parameters = {"criterion": ["gini","entropy"],
              "min_samples_split": [item for item in range(2,10,2)],
              "max_depth": [item for item in range(2,10,2)],
              "min_samples_leaf": [item for item in range(1,5)],
              "max_leaf_nodes": [None] + [item for item in range(2,10,2)],
              "random_state" : [1]
              }

dt = DecisionTreeClassifier()
gridsearch_dt = GridSearchCV(dt, parameters, cv=5)
gridsearch_dt.fit(X_train,y_train)
print(gridsearch_dt.best_params_)
print("Best score from cross validation: {0}".format(gridsearch_dt.best_score_))

{'criterion': 'gini', 'max_depth': 4, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 1}
Best score from cross validation: 0.9666666666666667


# Activity

Use cross validation to find optimal depth for decision trees

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV

In [30]:
# read data
df = pd.read_csv('~/Dropbox/packt/snakes_edit.csv')

# transform data to design matrix
X_total = df.drop(['Venom_ind','Venom'],axis=1)
y = df['Venom_ind']

X_train, X_test, y_train, y_test = train_test_split(X_total, y, test_size=0.33, random_state=100)

# scale numeric data to have zero mean and unit variance
X_scaled_temp = X_train.loc[:,['Length','Diameter','Rattle','Food','Predator','Scales Per Inch']]
scaler = StandardScaler().fit(X_scaled_temp)
X_scaled_temp = scaler.transform(X_scaled_temp)
X_train.loc[:,['Length','Diameter','Rattle','Food','Predator','Scales Per Inch']] = X_scaled_temp

X_scaled_test = X_test.loc[:,['Length','Diameter','Rattle','Food','Predator','Scales Per Inch']]
X_scaled_temp = scaler.transform(X_scaled_test)
X_test.loc[:,['Length','Diameter','Rattle','Food','Predator','Scales Per Inch']] = X_scaled_temp


(10000, 17)


/home/kiran/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/kiran/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [32]:
dt = DecisionTreeClassifier(max_depth=4,
                                random_state=100)
dt.fit(X_train, y_train)
print(dt.score(X_test,y_test))
dt

0.6818181818181818


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [18]:
# plot decision tree using graphviz
from sklearn import tree
import graphviz
dot_data = tree.export_graphviz(dt, out_file=None)
graph = graphviz.Source(dot_data)
graph.format = 'png'
graph.render("dtree-before")

'dtree-before.png'

In [39]:
parameters = {"criterion": ["gini"],
              "min_samples_split": [item for item in range(2,10,2)],
              "max_depth": [item for item in range(2,10,2)],
              "min_samples_leaf": [item for item in range(1,5)],
              "max_leaf_nodes": [None] + [item for item in range(2,10,2)],
              "random_state" : [100]
              }
dt = DecisionTreeClassifier()
gridsearch_dt = GridSearchCV(dt, parameters, cv=5)
gridsearch_dt.fit(X_train,y_train)
print(gridsearch_dt.best_score_)
print(gridsearch_dt.best_params_)

{'criterion': 'gini', 'max_depth': 8, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 4, 'random_state': 100}
0.6794029850746268


In [37]:
print("\n\n-- Testing best parameters [Grid]...")
dt_gridsearch = gridsearch_dt.best_estimator_
dt_gridsearch.fit(X_train, y_train)
print(dt_gridsearch.score(X_test,y_test))
print(dt_gridsearch)



-- Testing best parameters [Grid]...
0.6845454545454546
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')


In [38]:
# plot decision tree using graphviz
from sklearn import tree
import graphviz
dot_data = tree.export_graphviz(dt_gridsearch, out_file=None)
graph = graphviz.Source(dot_data)
graph.format = 'png'
graph.render("dtree-afterCV")

'dtree-afterCV.png'